# Asymmetry Case Break Downs

In [92]:
from py2neo import Node, Relationship, Graph
from numpy.random import rand
import numpy as num
from bokeh.plotting import output_notebook,figure, show
from bokeh.layouts import row,column,gridplot
from bokeh.models import Label
import numpy as np
from curve import *
from bokeh.models import Span,Legend
output_notebook()

Loading BokehJS ...

In [93]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [94]:
import pickle as pk
A = pk.load(open('local.pk','rb'))


import random

def gen_hex_colour_code():
   return '#'+''.join([random.choice('0123456789ABCDEF') for x in range(6)])

C = {}

In [95]:
R = roi_regressors()

/Users/kshadi3/Desktop/MANIA2/sample/curve.py:271: RuntimeWarning: invalid value encountered in double_scalars
  ce = num.sum([xx[1] for xx in A[i]['envelope']])/len(A[i]['envelope'])
/Users/kshadi3/Desktop/MANIA2/sample/curve.py:272: RuntimeWarning: invalid value encountered in double_scalars
  cex = num.sum([xx[0] for xx in A[i]['envelope']])/len(A[i]['envelope'])
/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [96]:
D = all_regressors(R,A)

/Users/kshadi3/Desktop/MANIA2/sample/curve.py:319: RuntimeWarning: invalid value encountered in double_scalars
  fx = (sl[0]*r[0]+(100-r[0])*tmp)/100


In [109]:
A[1]['n1']

'L1'

In [110]:
l = len(A)
A_dic = {}
for i in range(l):
    tmp = A[i]
    n1 = tmp['n1']
    n2 = tmp['n2']
    A_dic[n1+n2] = i

In [137]:
def roi_rot(w):
    p = w.split('L')
    return 'L'+p[-1]+'L'+p[1]
def logit(t):
    tmp = max(1,t)
    return num.log(tmp/5000.0)

In [138]:
null = {}
for i in range(1,181):
    for j in range(1,181):
        if (i==j):continue
        ind = A_dic['L'+str(i)+'L'+str(j)]
        tmp = A[ind]['weight']
        tmp = logit(tmp)
        if (tmp < -4.605170185988091):
            null['L'+str(i)+'L'+str(j)] = 1
        else:
            null['L'+str(i)+'L'+str(j)] = 0

In [162]:
total = sum(null.values()) # total null connections
asym = 0 # total asymmetric null connections
raw = [] #asymmtric case max weights in both directions
for w in null.keys():
    if null[w] == 1 and null[roi_rot(w)] == 0:
        asym = asym+1
        ind1 = A_dic[w]
        t1 = A[ind1]['weight']
        s1 = logit(t1)
        ind2 = A_dic[roi_rot(w)]
        t2 = A[ind2]['weight']
        s2 = logit(t2)
        raw.append((s1,s2))

# Case 1: Below the Noise floor in at least one direction
We call a connection below the noise floor a `null` connection

In [159]:
print('Total null connections:'+str(total))
print('Total null connections (percentage of all connections):'+str(total/32220))
print('Total asymmetric null connections:'+str(asym))
print('Total asymmetric null connections (percentage of all null connections):'+str(asym/total))

Total null connections:31575
Total null connections (percentage of all connections):0.9799813780260708
Total asymmetric null connections:563
Total asymmetric null connections (percentage of all null connections):0.017830562153602535


In [163]:
mp = list(map(lambda x:x[1],raw))
p1 = figure(plot_width=400, plot_height=400,title = 'Asummetry in null cases',y_axis_label = "Distribution",x_axis_label = "Opposite direction")
q1,q2 = np.percentile(mp,[0,100])

measured = [xx for xx in mp if (xx>q1 and xx<q2)]
hist, edges = np.histogram(measured, density=True, bins=50)

p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")
show(p1)

# CASE 2: 0-distance Connection

In [164]:
total = 0 # total adjacent connections
asym = 0 # total asymmetric adjacent connections
raw = [] #asymmtric case max weights in both directions
for w in null.keys():
    ind1 = A_dic[w]
    t1 = A[ind1]['a'][0]
    s1 = logit(A[ind1]['weight'])
    if t1<1:
        total = total + 1
        ind2 = A_dic[roi_rot(w)]
        s2 = logit(A[ind2]['weight'])
        t2 = A[ind2]['a'][0]
        if t1<1 and t2>1:
            asym = asym + 1
            raw.append((s1,s2))

In [165]:
print('Total 0-distance connections:'+str(total))
print('Total 0-distance connections (percentage of all connections):'+str(total/32220))
print('Total asymmetric 0-distance connections:'+str(asym))
print('Total asymmetric 0-distance connections (percentage of all null connections):'+str(asym/total))

Total 0-distance connections:31575
Total 0-distance connections (percentage of all connections):0.9799813780260708
Total asymmetric 0-distance connections:563
Total asymmetric 0-distance connections (percentage of all null connections):0.017830562153602535


In [166]:
mp = list(map(lambda x:abs(x[1]-x[0]),raw))
p1 = figure(plot_width=400, plot_height=400,title = 'Asummetry in 0-distance cases',y_axis_label = "Distribution",x_axis_label = "Difference in connection strength between directions (abs)")
q1,q2 = np.percentile(mp,[0,100])

measured = [xx for xx in mp if (xx>q1 and xx<q2)]
hist, edges = np.histogram(measured, density=True, bins=50)

p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")
show(p1)